In [49]:
from linkedin_scraper import Person, actions

import re
import time
from collections import defaultdict
 
import pandas as pd
import requests
from lxml import etree
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [50]:
driver = webdriver.Chrome()

#! 需要使用梯子登录，如果中途中断，则变换节点
email = "emails"
password = "passwords"
actions.login(driver, email, password) # if email and password isnt given, it"ll prompt in terminal

KeyboardInterrupt: 

In [45]:
names_to_search = [
    "Mira Murati", "Brad Lightcap", "Jason Kwon", "Wojciech Zaremba", "Alec Radford",
    "Anna Makanju", "Bob McGrew", "Srinivas Narayanan", "Che Chang", "Lillian Weng",
    "Mark Chen", "Sutskever", "John Schulman", "Barret Zoph", "Nick Ryder",
    "Alex Paino", "Miles Brundage", "Matt Knight", "Peter Welinder", "Hannah Wong",
    "Diane Yoon", "William Fedus", "Arka Dhar", "Jonathan Lachman", "Abe Land",
    "Aaron Hurst", "Abigail Raman", "Abram Wiesman", "Adam Goldberg", "Adam Goucher",
    "Adam Groth", "Adam Hupp", "Adam Nace", "Adam Perelman", "Aditya Ramesh", "Adrian Spânu",
    "Adrien Ecoffet", "Afsha Shaikh", "AJ Ostrow", "Ahmed El- Kishky", "Akila Welihinda",
    "Akshay Nathan", "Alex Baker", "Alex Beutel", "Alex Carney", "Alex Chow",
    "Alex Gee", "Alex Iftimie", "Alex Nichol", "Alex Renzin", "Alexander Karpenko",
    "Alexei Karpenko", "Alexander King", "Alexander Kirillov", "Alexandre Tachard Passos",
    "Alexandre", "Alexis Conneau", "Ali Kamali", "Alice Lee", "Aliisa Rosenthal",
    "Alison Harmon", "Alison McPhail", "Allan Jabri", "Allie Teague", "Allison Tam",
    "Avinash Jain", "Amanda Anderson", "Amanda Tenanes", "Amin Tootoonchian",
    "Amol Shah", "Ananya Kumari", "Andre Saraiva", "Andrea Vallone", "Andrew Braunstein",
    "Andrew Cann", "Andrew Kondrich", "Andrew Peng", "Andrew Top", "Andrew Tulloch",
    "Andrew YuXuan Liu", "Andrew Busker", "Andrey Mishchenko", "Andy Brown",
    "Angel Demirev", "Angela Jiang", "Angie Luo", "Anish Tondwalkar", "Ankush Gupta",
    "Anna- Luisa Brakman", "Anna McKean", "Anna Tifft", "Anthony Malinao", "Anton Tananaev",
    "Anushree Agrawal", "Arun Vijayvergiya", "Arvind Neelakantan", "Arzav Jain", "Ashley Pantuliano",
    "Ashvin Nair", "Atty Eleti", "Avi Nayak", "Avital Oliver", "Barak Mitchener",
    "Rebecca Waite", "Behrooz Ghorbani", "Belinda Truong", "Ben Leimberger", "Ben Rossen",
    "Ben Wang", "Benjamin Burrell", "Benjamin Sokolowsky", "Benjamin Zweig",
    "Bertrand Fan", "Bianca Martin", "Bill Peebles", "Bill Stavroulakis", "Billie Jonn",
    "Blake Samic", "Bob Rotsted", "Bobby", "Bogo Giertler", "Boris Power", "Bowen Baker",
    "Boyang Niu", "Brad Biegler", "Brandon Wang", "Brandon Houghton", "Brendan Herron",
    "Brendan Quinn", "Brittany Carey", "Brittany Smith", "Britton Jamison", "Brock Whittaker",
    "Brooke Chan", "Bryan Ashley", "Brydon Eastman", "Cadran Cowansage", "Caitlin Carroll",
    "Cameron Raymond", "Camillo Lugaresi", "Carl Ross", "Cary Bassin", "Casey Chu",
    "Chak Ming Li", "Chan Park", "Coley Czarnecki", "Corey Nayden", "Channing Conger",
    "Chelsea Carlson", "Chelsea Voss", "Chia Chung (Yoshi) Wang", "Chong Zhang", "Chris Berner",
    "Chris Brown", "Chris Clark", "Chris Finlayson", "Chris Haugli", "Chris Jiang", "Chris Rynerson",
    "Eric Sigler", "Erica Lee", "Erica Tom", "Eric Luhman", "Ehsan Asdar", "Ethan Stock",
    "Evan Lee", "Evan Morikawa", "Evan Weiss", "Faiz Munshi", "Farouk Hamzawi",
    "Fan Wang", "Farzad Khorasani", "Felipe Such", "Felipe Torres", "Fereshte Khani",
    "Filipe de Avila Belbute Peres", "Filippo Raso", "Florencia Leoni Aleman", "Foivos Tsimpourlas",
    "Fotios Chantzis", "Francis Real", "Francis Song", "Francis Zhang", "Fred von Lohmann",
    "Gabriel Goh", "Garrett McCarthy", "Gene Oden", "Geoffrey Salmon", "Giambattista Parascandolo",
    "Gideon Myles", "Gildas Chabot", "Glory Jain", "Gretchen Krueger", "Hadi Salman",
    "Haiming Bao", "Haitang Hu", "Haozhun Jin", "Harold Nguyen", "Hart Andrin", "Harri Edwards",
    "Heather Schmidt", "Heewoo Jun", "Henk Tillman", "Henri Roussez", "Henrique Ponde de Oliveira Pinto",
    "Henry Head", "Hessam Bagherinezhad", "Hongyu Ren", "Huiwen Chang", "Hugh Drinkwater",
    "Hyung Won Chung", "Hyunil(Chester) Cho", "Hunter Lightman", "Hyeonwoo Noh", "Ian Kivlichan",
    "Ian Mobbs", "Ian O'Connell", "Ian Osband", "Ian Silber", "Ibrahim Okuyucu", "Lan Akkay",
    "Lan Kostrikov", "Ingmar Kanitscheider", "Irwan Bello", "Isa Fulford", "Isabella D'Agostino",
    "Ishaan Gulrajani", "Jackie Hehir", "(Jan) Hendrik Kirchner", "Jacob Menick", "Jake Brill",
    "Jake McNeil", "James Betker", "Jamie Caulkins", "James Dyett", "James Hills", "Jamie Kiros",
    "Janine Korovesis", "Jarad Gilmore", "James Hairston", "Janice Kwong", "Jason Teplitz",
    "Jason Tunzini", "Jason Wei", "Jason Wolfe", "Javier Soto", "Jean Harb", "Jean Neaylon",
    "Jeff Belgum", "Jeff Chan", "Jeff Harris", "Jelani Robinson", "Jenia Varavva", "Jeremiah Currier",
    "Jerry Tworek", "Jessica James", "Jessica Lee", "Jessica Shieh", "Jiahui Yu", "Jiayi Weng", "Jie Tang", "Jillian Khoo", "Joanne Jang", "Jodi Kean", "Joe Gillespie", "Joe Palermo",
    "Joe Taylor", "Joel Parish", "Joey Flynn", "Johannes Heidecke", "John Allard", "John Lam", "John Rizzo", "Jonathan Ward", "Jonathan Becton", "Jonathan Gordon", "Jonathan McKay",
    "Jonathan Reiber", "JongWook Kim", "Jordan Sitkin", "Jordan Williams", "Jos Kraaijeveld", "Joselyn Jauregui", "Joseph Landers", "Joseph Millman", "Josh Gross", "Josh Kaplan", "Josh Snyder","Joshua Achiam", "Joy Cashman", "Joyce Lee", "Juan Red Avila", "Juntang Zhuang", "Julia Collins", "Julian Le", "Julius Thrun", "Junjie Huang", "Kai Chen",
    "Kai Fricke", "Kai Hayashi", "Kai Xiao", "Kaitlin Gottfried", "Karen Toro",
    "Karolis Kosas", "Karl Cobbe", "Karthik Rangarajan", "Kathy Son", "Katy Elkin",
    "Kayla Wood", "Kendra Rimbach", "Ken Langlinais", "Kenny Hsu", "Kevin Button",
    "Kevin Gladstone", "Kevin Liu", "Kevin Metti", "Kevin Park", "Kevin Yu",
    "Kiel Howe", "Kim Malfacini", "Krissi Robinson", "Kristen Dallara", "Krithika Muthukumar", "Kyla Sheppard", "Kyle Miller", "Kyle Luther", "Lane Dilg", "Lama Ahmad",
    "Larry Kai", "Larry Xu", "Laura Wang", "Lauren Itow", "Lauren Workman", "Lee Grimes",
    "Lee Wei An", "Leigh Ann Montano", "Lenny Bogdonoff", "Li Jing", "Lia Guy",
    "Lien Mamitsuka", "Lin Yang", "Lindsey Held", "Lisa Pan", "Logan Kilpatrick",
    "Louis Feuvrier", "Lukasz Kaiser", "Lukasz Kondraciuk", "Luke Metz", "Luke Miller",
    "Mada Aflak", "Maddie Casey", "Maddie Simens", "Madeline Casey", "Maggie Hott",
    "Manas Joglekar", "Marat Dukhan", "Marc Manara", "Marco Noce", "Mario Saltarelli",
    "Marko Tintor", "Mark Hudnall", "Marvin Zhang", "Mason Meyer", "Mateusz Litwin",
    "Matt Benice", "Matt Beutler", "Matt Castle", "Matt Chung", "Matt Hartill",
    "Matt Yoshida", "Matthew Lim", "Max Reid", "Meghan Shah", "Mehmet Yatbaz",
    "Mengyuan Yan", "Michael Flossman", "Michael Janner", "Michael Kolhede",
    "Michael Lampe", "Michael Petrov", "Mikhail Pavlov", "Vitchyr H E Pong", "Zahi  Moudallal", "Miguel Castro", "Niko Felix", "Nicolai Skabo", "Yonadav  Shavit", "Scott Gray", "Pavel lzmailov", "Rowan Zellers", "Suchir Balaji", "Nick Cooper", "Ryan  Ragona", "Sean Murphy", "Ryan Hartnett", "Vanessa Gatihi", "Steven Bills", "Molly Lin", "Rosie Campbell", "Warren Ouyang", "Natalie Summers", "Renaud Gaubert", "Rohit Ramchandani", "Sam Schoenholz", "Michelle Pokrass", "Theresa Lopez", "Paul  McMillan", "Yash  Patil", "Patryk Lesiewicz", "Tim Brooks", "Trevor Mavrakis", "Nick Hamilton", "Stephen Petersilge", "Ryan Biddy", "Natalie Cone", "Rhythm Garg", "RJ Marsan", "Rohan Sahai", "Shino Jomoto", "Noah Deutsch", "Philippe Tillet", "Sean Grove", "Tal   Stramer", "Yujia Jin", "Tal Broda", "Mostafa Rohaninejad", "Michael Trinh", "Philip Su", "Natalia Gimelshein", "Michael Wu", "Prasad Chakka", "Nickolas Stathas", "Jp  Villafuerte", "Michelle Zheng", "Tiffany Citra", "Mollie Javerbaum", "Wenlei Xie", "Robert Infantino", "Randall Lin", "Renny Hwang", "Tabarak Khan", "Ryan Lowe", "Todd Underwood", "Qiming Yuan", "Yasu Sakamoto", "Roshan James", "Romain  Huet", "Sandhini Agarwal", "Yuchen  He", "Miguel Manriquez", "Shengli Hu", "Roy Ziv", "Peter X Deng", "Trevor Cai", "Rachel Lee", "Sarah Burns", "Tarun Gogineni", "Natalie Kim", "Michael Schade", "Shaun Van Weelden", "Tom Stasi", "Michelle Kim", "Sherwin Wu", "Reza Zamani", "Youlong  Cheng", "Nicholas Martin", "Sean Metzger", "Tatiana Zolotova", "Katie Mayer", "Sabrina Dermody", "Simon Posada Fishman", "Rahul Arora", "Ugurcan Turkdogan", "Tony Wu", "Yilei  Qian", "Taylor Gordon", "Vishal   Kuo", "Wesam  Manassra", "Ryan Greene", "Richard Ngo", "Mohammad Bavarian", "Troy Peterson", "Paul Punia", "Oleg Boiko", "Todor Markov", "Rasmus Rygaard", "Pei-Chee Juang", "Yu Zhang", "Raul Puri", "Steven Adler", "Rod  Solaimani", "Nick Kartashov", "Ryan Peterson", "Richard Ho", "Tomer Kaftan", "Valerie  Balcom", "Victoria  Spiegel", "Peter Zhokhov", "Tifa Chen", "Tyna  Eloundou", "Peter Dolan", "Rory Carmichael", "Olivier Godement", "Phong Nguyen", "Tianhao Zheng", "Prafulla  Dhariwal", "Natalie Staudacher", "Xinyang   Geng", "Trevor Hildenberger", "Stewart Hall", "Radhika Mathur", "Morgan Griffiths", "Yang  Song", "Tao Xu", "Scott Lessans", "Shibani Santurkar", "Yongjik  Kim", "Steven Heidel", "Yury Malkov", "Nick Turley", "Tao    Wang", "Yuchen  Zhang", "Reiichiro Nakano", "Will DePue", "Tejal Patwardhan", "Rob Honsby", "Will  Saborio", "William Butler", "Pedram Keyani", "Orla McHenry", "Sowmya Ranganathan", "R|ajeev Nayak", "Toki   Sherbakov", "Peter Hoeschele", "Paul Baltescu", "Shirong Wu", "Tong Mu", "Trapit Bansal", "Richard Hasslacher", "Pritesh  Badiani", "Stephanie Lin", "Yining  Chen", "Noam Brown", "Teddy Lee", "Morgan Grafstein", "Owen Campbell-Moore", "Rachel Yun Shi Lim", "Ryan Cheu", "Samuel Wolrich", "Stacie Faggioli", "Zhengwen   Zhou", "Patrik Goethe", "Sarah Shoker", "Sheila Dunning", "Zhi  Bie", "Naga Sai Anup Krishnamraju", "Vinnie  Monaco", "Pia Santos", "Xuanhan(Eimi)Zhang", "Nik Tezak", "Shantanu Jain", "Michelle Fradin", "Stephanie Tran", "Reggie  Hall", "Shengjia Zhao", "Nikunj Handa", "Paul Shen", "Vineet  Kosaraju", "Wade Morgan", "Sandro Gianella", "Tom Dupre la Tour", "Thomas Dimson", "Ming Chen", "Yunxing  Dai", "Scott Mayer McKinney", "Philip  Pronin", "Troy Luhman", "Nitish Shirish Keskar", "Mike Heaton", "Oleg Murk", "Patricia Lue", "Sherry Lachman", "Yunxin Joy Jiao", "Sophie Rose", "Rocky Smith", "Thomas Degry", "Stefanie Biaggi", "Thomas Raoux", "Tom Rubin", "Robert Pulford", "Sarah Yoo", "Sully Chen", "Pamela Mishkin", "Noel Bundick", "Ola Okelola", "Neal Chuenprateep", "Saravpreet Hira", "Shuyuan Zhang", "Siyuan Fu", "Noah Jorgensen", "Parijat Talkad", "Rapha Gontijo Lopes", "Yutian Liu", "Mike Hunter", "Victoria  Lampton", "Yufei  Guo", "Owen Lin", "Wenda Zhou", "Nick Lee", "Spencer Papay", "Sriya Santhanam", "Rui Shu", "Xin  Hu", "Nikki Sommer", "Shyamal Anadkat"]


# 用于测试功能
names_to_search_test = ["Pamela Mishkin","kkk", "Owen Lin"]

In [46]:
# def onetime(text):
#     # 将文本拆分成行
#     lines = text.split("\n")

#     # 使用 set 去除重复行
#     unique_lines = set(lines)

#     # 将去重后的行拼接成字符串
#     result_text = "\n".join(unique_lines)
#     return result_text


def remove_consecutive_duplicates(input_str):
    lines = input_str.strip().split('\n')
    unique_lines = [lines[0]] + [lines[i] for i in range(1, len(lines)) if lines[i] != lines[i-1]]
    result_str = '\n'.join(unique_lines)
    return result_str

In [47]:
person=[]

# 进入LinkedIn首页
driver.get("https://www.linkedin.com/feed/")

i=0

for name in names_to_search:
    i=i+1
    
    # 找到搜索框
    search_box = driver.find_element(By.XPATH, "/html/body/div[5]/header/div/div/div/div[1]/input")
    
    # 清空搜索框
    search_box.clear()

    # 输入人名
    search_box.send_keys(name+" openai")
    search_box.send_keys(Keys.ENTER)

    # 等待搜索结果加载
    time.sleep(3)

    # 找到搜索结果中的用户链接，点击进入用户页面
    # 找到搜索结果中的用户链接，点击进入用户页面
    try:
        user_link = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[1]/div/ul/li/div/a/div/div[1]/div[1]/div/div/span[1]/span/a/span/span[1]")
        user_link.click()
        
        # 等待用户页面加载
        time.sleep(3)

        # 在这里可以对用户页面进行爬取操作，例如获取用户信息、连接数量等.
        # 这里可以看看其他人的代码是怎么爬取的
        person_info = []

        # Re-locate the elements before interacting with them
        name_element = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[1]/span/a/h1")
        title_element = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[2]")
        # location_element = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[1]")
        # Get text from the located elements
        name = name_element.text
        title = title_element.text
        # location = location_element.text
        
        work_company_all=[]
        # education_all=[]
        #! 失败的原因：section报错检索不到，需要一个可以一直循环检索section的方法。
        for k in range(3,8):
            try:
                # work_company_element = driver.find_element(By.XPATH, f"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[{k}]/div[3]/ul/li/div")
                # /html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]/div[3]/ul/li[2]/div
                # work_company_element = driver.find_element(By.XPATH, f"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[{k}]/div[3]/ul")
                work_company_element = driver.find_element(By.XPATH, f"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[{k}]/div[3]/ul")
                # /html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]
                work_company = work_company_element.text
                work_company=remove_consecutive_duplicates(work_company)
                work_company_all.append(work_company)
            except Exception as e:
                print(f"Error while processing {name}: {e}")   
            # try:
            #     education_element = driver.find_element(By.XPATH, f"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[{i}]")
            #     # /html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]
            #     # /html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]
            #     education = education_element.text
            #     education=onetime(education)
            #     education_all.append(education)
            # except Exception as e:
            #     print(f"Error while processing {name}: {e}")   


        print("name:", name)
        print("title:", title)
        # print("location:", location)
        print("work_company_all:", work_company_all)
        # print("education_all:", education_all)

        person_info.append(name)
        person_info.append(title)
        # person_info.append(location)
        person_info.append(work_company_all)
        # person_info.append(education_all)
        person.append(person_info)
        
        print("数量",i)
        # 每隔20轮保存一次
        if i%20==0:
            # 转换为DataFrame
            df = pd.DataFrame(person, columns=['姓名', '职位','工作/教育经历'])

            # 将DataFrame写入Excel文件
            df.to_excel(f'D:/Python/SCGC/linkedin/person_{i}.xlsx', index=False)
            print("--------------成功保存--------------")

        # 返回到搜索结果页面
        driver.back()
        time.sleep(3)

    except Exception as e:
        print(f"Error while processing {name}: {e}")

print("person:",person)
# 关闭浏览器
# driver.quit()

KeyboardInterrupt: 